In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import keras_tuner as kt


### Define functions for use during optimization

In [2]:
def reduce_count_vals(df, colname, threshold):
    # Determine which values to replace if counts are less than ...?
    counts = df[colname].value_counts()
    replace_list = list(counts[counts < threshold].index)

    # Replace in dataframe
    for item in replace_list:
       df[colname] = df[colname].replace(item,"Other")
    
    # Check to make sure binning was successful
    df[colname].value_counts()

In [3]:
def do_one_hot(df, y_col):
    # Create a list of columns that are 'object' type
    obj_cat = df.dtypes[df.dtypes == "object"].index.tolist()
    # Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)
    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(df[obj_cat]))
    # Add the encoded variable names to the DataFrame
    encode_df.columns = enc.get_feature_names(obj_cat)
    
    # Merge one-hot encoded features and drop the originals
    df = df.merge(encode_df, left_index=True, right_index=True)
    df = df.drop(obj_cat,1)
    
    # Split our preprocessed data into our features and target arrays
    y = df[y_col].values
    df.drop(columns=[y_col], inplace=True)
    X = df.values
    print(f"merged df.shape()={df.shape}")
    
    return df, X, y

In [4]:
def do_scatter_plots(df):
    for xx in ['APPLICATION_TYPE', 
        'AFFILIATION', 
        'CLASSIFICATION', 
        'USE_CASE', 
        'ORGANIZATION', 
        'INCOME_AMT', 
              ]:
        for yy in ['APPLICATION_TYPE', 
            'AFFILIATION', 
            'CLASSIFICATION', 
            'USE_CASE', 
            'ORGANIZATION', 
            'INCOME_AMT'
                  ]:
            if xx == yy:
                continue
            titlestring = f"x={xx} vs. y={yy}"
            df.plot.scatter(x=xx, y=yy, title=titlestring, c='IS_SUCCESSFUL', colormap='winter')


In [5]:
def do_scale(scaler):
    # Scale the data
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    return X_train_scaled, X_test_scaled

In [6]:
def build_model(inputs, layers):
    nn = tf.keras.models.Sequential()
    first = True
    for layer in layers:
        if first:
            first = False
            nn.add(tf.keras.layers.Dense(units=layer['units'], activation=layer['act'], input_dim=inputs))
        else:
            nn.add(tf.keras.layers.Dense(units=layer['units'], activation=layer['act']))

    print(nn.summary())
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn

In [7]:
def train_nn_model(model, X_train, y_train, n_epochs, checkpoint_dir):
    # Create a callback that saves the model's weights every epoch
    from tensorflow.keras.callbacks import ModelCheckpoint
    
    # Define the checkpoint path and filenames
    os.makedirs("checkpoints_opt",exist_ok=True)
    #checkpoint_file = f"weights.{epoch:02d}.hdf5"
    #checkpoint_path = f"checkpoints_opt/weights.{epoch:02d}.hdf5"
    checkpoint_path = "checkpoints_opt/weights_2.{epoch:02d}.hdf5"
    
    cp_callback_opt = ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=1,
        save_weights_only=True,
        period=5)

    # Normally we use 'save_freq', but it behaves strangely, and I could not get
    # it to save every 5 epochs. The 'period' param is now deprecated, but it works.
        #save_freq='epoch')
        
    fit_model = model.fit(X_train, y_train, epochs=n_epochs, callbacks=[cp_callback_opt])
    return fit_model

### Optimation Run 1 - With 7 layers of tanh & sigmoid

In [ ]:
application_df = pd.read_csv("Resources/charity_data.csv")
csv_cols = [
    'EIN', 
    'NAME', 
    'APPLICATION_TYPE', 
    'AFFILIATION', 
    'CLASSIFICATION', 
    'USE_CASE', 
    'ORGANIZATION', 
    'STATUS', 
    'INCOME_AMT', 
    'SPECIAL_CONSIDERATIONS', 
    'ASK_AMT', 
    'IS_SUCCESSFUL'
]
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

reduce_count_vals(application_df, 'APPLICATION_TYPE', 500)
reduce_count_vals(application_df, 'CLASSIFICATION', 1500)

application_df, X, y = do_one_hot(application_df, 'IS_SUCCESSFUL')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train_scaled, X_test_scaled = do_scale(StandardScaler())
#X_train_scaled, X_test_scaled = do_scale(MinMaxScaler())

nins = application_df.shape[1]
print(f"nins={nins}")
nn_model = build_model(inputs=nins, layers=[
    {'units': 340, 'act': 'tanh'},
    {'units': 170, 'act': 'tanh'},
    {'units': 85,  'act': 'tanh'},
    {'units': 40,  'act': 'tanh'},
    {'units': 20,  'act': 'sigmoid'},
    {'units': 5,   'act': 'sigmoid'},
    {'units': 1,   'act': 'sigmoid'}
])
trained_model = train_nn_model(nn_model, X_train_scaled, y_train, n_epochs=500, checkpoint_dir="checkpoints_opt")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_opt1.h5")


merged df.shape()=(34299, 43)
nins=43
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 340)               14960     
_________________________________________________________________
dense_1 (Dense)              (None, 170)               57970     
_________________________________________________________________
dense_2 (Dense)              (None, 85)                14535     
_________________________________________________________________
dense_3 (Dense)              (None, 40)                3440      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 105       
_________________________________________________________________
dense_6 (Dense)   

C:\Users\bjile\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


Epoch 1/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5924 - accuracy: 0.7132
Epoch 2/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5672 - accuracy: 0.7293
Epoch 3/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5603 - accuracy: 0.7294
Epoch 4/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5567 - accuracy: 0.7306
Epoch 5/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7313

Epoch 00005: saving model to checkpoints_opt\weights_2.05.hdf5
Epoch 6/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5531 - accuracy: 0.7310
Epoch 7/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5505 - accuracy: 0.7332
Epoch 8/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5511 - accuracy: 0.7323
Epoch 9/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7320
Epoch 10/500
804/804 [=====

804/804 [==============================] - 2s 2ms/step - loss: 0.5349 - accuracy: 0.7414
Epoch 73/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5357 - accuracy: 0.7407
Epoch 74/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5353 - accuracy: 0.7401
Epoch 75/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5348 - accuracy: 0.7410

Epoch 00075: saving model to checkpoints_opt\weights_2.75.hdf5
Epoch 76/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5341 - accuracy: 0.7410
Epoch 77/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5347 - accuracy: 0.7411
Epoch 78/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5348 - accuracy: 0.7405
Epoch 79/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7410
Epoch 80/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5347 - accuracy: 0.7411

Epoch 00080: saving model to c

804/804 [==============================] - 2s 2ms/step - loss: 0.5336 - accuracy: 0.7420
Epoch 144/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5331 - accuracy: 0.7419
Epoch 145/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5332 - accuracy: 0.7423

Epoch 00145: saving model to checkpoints_opt\weights_2.145.hdf5
Epoch 146/500
804/804 [==============================] - 2s 3ms/step - loss: 0.5334 - accuracy: 0.7417
Epoch 147/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7422
Epoch 148/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7419
Epoch 149/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5325 - accuracy: 0.7426
Epoch 150/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5326 - accuracy: 0.7416

Epoch 00150: saving model to checkpoints_opt\weights_2.150.hdf5
Epoch 151/500
804/804 [==============================] - 2s 

### Optimization Run 2 - Same as Run1, but drop 'STATUS' & 'SPECIAL_CONSIDERATIONS'; and bucketize 'AFFILIATION', 'ORGANIZATION', AND 'INCOME_AMT'

In [ ]:
application_df = pd.read_csv("Resources/charity_data.csv")
csv_cols = [
    'EIN', 
    'NAME', 
    'APPLICATION_TYPE', 
    'AFFILIATION', 
    'CLASSIFICATION', 
    'USE_CASE', 
    'ORGANIZATION', 
    'STATUS', 
    'INCOME_AMT', 
    'SPECIAL_CONSIDERATIONS', 
    'ASK_AMT', 
    'IS_SUCCESSFUL'
]

application_df.drop(columns=['EIN', 'NAME','STATUS', 'SPECIAL_CONSIDERATIONS'], inplace=True)

reduce_count_vals(application_df, 'APPLICATION_TYPE', 500)
reduce_count_vals(application_df, 'CLASSIFICATION', 1500)
reduce_count_vals(application_df, 'AFFILIATION', 100)
reduce_count_vals(application_df, 'ORGANIZATION', 500)
reduce_count_vals(application_df, 'INCOME_AMT', 500)

application_df, X, y = do_one_hot(application_df, 'IS_SUCCESSFUL')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
#X_train_scaled, X_test_scaled = do_scale(StandardScaler())
X_train_scaled, X_test_scaled = do_scale(MinMaxScaler())

nins = application_df.shape[1]
print(f"nins={nins}")
nn_model = build_model(inputs=nins, layers=[
    {'units': 340, 'act': 'relu'},
    {'units': 170, 'act': 'relu'},
    {'units': 85,  'act': 'relu'},
    {'units': 40,  'act': 'relu'},
    {'units': 20,  'act': 'sigmoid'},
    {'units': 5,   'act': 'sigmoid'},
    {'units': 1,   'act': 'sigmoid'}
])
trained_model = train_nn_model(nn_model, X_train_scaled, y_train, n_epochs=500, checkpoint_dir="checkpoints_opt")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_opt2.h5")

### Optimization Run 3 - with KerasTuner

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options

def create_tuner_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=70,
        max_value=140,
        step=10), activation=activation, input_dim=40))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for ii in range(hp.Int('num_layers', 1, 10)):
        #nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(ii),
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(ii),                                             
            min_value=70,
            max_value=140,
            step=10),
            activation=activation))
            
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
def run_keras_tuner(X_train, y_train, n_epochs, validation_data):
    # Create a `Hyperband()` tuner instance
    tuner = kt.Hyperband(
        create_tuner_model,
        objective="val_accuracy",
        max_epochs=50,
        hyperband_iterations=2,
        overwrite=True)

    # Run the kerastuner search for best hyperparameters
    tuner.search(X_train, y_train, epochs=n_epochs, validation_data=validation_data)
    return tuner

In [ ]:
application_df = pd.read_csv("Resources/charity_data.csv")
csv_cols = [
    'EIN', 
    'NAME', 
    'APPLICATION_TYPE', 
    'AFFILIATION', 
    'CLASSIFICATION', 
    'USE_CASE', 
    'ORGANIZATION', 
    'STATUS', 
    'INCOME_AMT', 
    'SPECIAL_CONSIDERATIONS', 
    'ASK_AMT', 
    'IS_SUCCESSFUL'
]

application_df.drop(columns=['EIN', 'NAME','STATUS', 'SPECIAL_CONSIDERATIONS'], inplace=True)

reduce_count_vals(application_df, 'APPLICATION_TYPE', 500)
reduce_count_vals(application_df, 'CLASSIFICATION', 1500)

application_df, X, y = do_one_hot(application_df, 'IS_SUCCESSFUL')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train_scaled, X_test_scaled = do_scale(StandardScaler())
#X_train_scaled, X_test_scaled = do_scale(MinMaxScaler())

nins = application_df.shape[1]
print(f"nins={nins}")

tuner = run_keras_tuner(X_train_scaled, y_train, n_epochs=20, validation_data=(X_test_scaled,y_test))

# # Evaluate the model using the test data
# model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_opt3.h5")

In [ ]:
# Get top 3 model hyperparameters and print the values
best_hypers = []
for ii in range(1,4):
    best_hyper = tuner.get_best_hyperparameters(ii)[0]
    print(best_hyper.values)
    best_hypers.append(best_hyper)

In [ ]:
# Evaluate the top 3 models against the test dataset
best_models = []
for ii in range(1,4):
    best_model = tuner.get_best_models(ii)[0]
    model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    best_models.append(best_model)